In [ ]:
#|hide
#|skip
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

convert pip wheel -> conda package

whl   zip
 - contains cymem-2.0.2.dist-info : LICENSE  METADATA  RECORD  top_level.txt  WHEEL

conda tgz
 - includes lib/python{v}/site-packages/
 - contains info : "about.json  files  git  hash_input.json  index.json  paths.json"

conversion:

1. move main modules to lib/...
2. create info

---

- about: home license summary
- files: list
- git: empty
- hash_input: empty
- index.json: create
- paths: files with sha256 and size

In [ ]:
from fastai.torch_basics import *
import zipfile,tarfile
from email.parser import Parser
from wheel import wheelfile

In [ ]:
from conda.gateways.disk.read import read_python_record

In [ ]:
path = Path('~/git/spacy_conda/cymem').expanduser()
pkg = path/'pkg/cymem-2.0.2-py37_0.tar.bz2'
whl = path/'whl/cymem-2.0.2-cp37-cp37m-manylinux1_x86_64.whl'
pkg.exists(),whl.exists()

(True, True)

In [ ]:
zpkg = tarfile.open(pkg,"r:bz2")

In [ ]:
lpkg = zpkg.getmembers()
[(i,o.name) for i,o in enumerate(lpkg) if o.name.startswith('info/')]

[(0, 'info/hash_input.json'),
 (1, 'info/index.json'),
 (2, 'info/files'),
 (3, 'info/paths.json'),
 (4, 'info/about.json'),
 (5, 'info/git')]

In [ ]:
f = (zpkg.extractfile('info/about.json').read().decode())
print(f)
fn.name

{
  "channels": [
    "pytorch",
    "fastai",
    "defaults"
  ],
  "conda_build_version": "3.18.8",
  "conda_private": false,
  "conda_version": "4.7.12",
  "env_vars": {
    "CIO_TEST": "<not set>"
  },
  "extra": {
    "copy_test_source_files": true,
    "final": true
  },
  "home": "https://github.com/explosion/cymem",
  "identifiers": [],
  "keywords": [],
  "license": "MIT",
  "root_pkgs": [
    "partd 1.0.0 py_0",
    "nltk 3.4.5 py37_0",
    "beautifulsoup4 4.8.0 py37_0",
    "sphinxcontrib-htmlhelp 1.0.2 py_0",
    "wheel 0.33.6 py37_0",
    "mock 3.0.5 py37_0",
    "numpy-base 1.17.2 py37hde5b4d6_0",
    "qtpy 1.9.0 py_0",
    "mistune 0.8.4 py37h7b6447c_0",
    "bitarray 1.0.1 py37h7b6447c_0",
    "libtiff 4.0.10 h2733197_2",
    "mpc 1.1.0 h10f8cd9_1",
    "cryptography 2.7 py37h1ba5d50_0",
    "pytest-astropy 0.5.0 py37_0",
    "krb5 1.16.1 h173b8e3_7",
    "libssh2 1.8.2 h1ba5d50_0",
    "anaconda custom py37_1",
    "ipython 7.8.0 py37h39e3cac_0",
    "zlib 1.2.11 h7b64

'info/index.json'

In [ ]:
fwhl = wheelfile.WheelFile(whl)

In [ ]:
fwhl.parsed_filename.groupdict()

{'namever': 'cymem-2.0.2',
 'name': 'cymem',
 'ver': '2.0.2',
 'build': None,
 'pyver': 'cp37',
 'abi': 'cp37m',
 'plat': 'manylinux1_x86_64'}

In [ ]:
fwhl.dist_info_path

'cymem-2.0.2.dist-info'

In [ ]:
fwhl.record_path

'cymem-2.0.2.dist-info/RECORD'

In [ ]:
read_python_record(whl.parent, fwhl.record_path, '3.7')['paths_data'].paths

(PathDataV1(_path='lib/python3.7/site-packages/cymem-2.0.2.dist-info/LICENSE', path_type='hardlink', sha256='ednboUtKqeqeBouyYb59GvQN9K04gEyyw2KwK5cPxHo', size_in_bytes=1139),
 PathDataV1(_path='lib/python3.7/site-packages/cymem-2.0.2.dist-info/METADATA', path_type='hardlink', sha256='9tePG2KbFvrQ1Sy86GeZfihXDryyKcI1jS_7ZDcrTjc', size_in_bytes=8585),
 PathDataV1(_path='lib/python3.7/site-packages/cymem-2.0.2.dist-info/RECORD', path_type='hardlink', sha256=None, size_in_bytes=None),
 PathDataV1(_path='lib/python3.7/site-packages/cymem-2.0.2.dist-info/WHEEL', path_type='hardlink', sha256='QNlK5ivAimyKC6GBwwJ1xZo1FLZqOgbKwJoOKciA_fE', size_in_bytes=109),
 PathDataV1(_path='lib/python3.7/site-packages/cymem-2.0.2.dist-info/top_level.txt', path_type='hardlink', sha256='dCYwCFWTj_NE7Z9zFrrKqAcWuUCtg52zKRI4l_Wrr7Q', size_in_bytes=6),
 PathDataV1(_path='lib/python3.7/site-packages/cymem/__init__.pxd', path_type='hardlink', sha256='47DEQpj8HBSa-_TImW-5JCeuQeRkm5NMpJWZG3hSuFU', size_in_bytes=0),

In [ ]:
fwhl.namelist()

['cymem/cymem.pxd',
 'cymem/cymem.pyx',
 'cymem/__init__.py',
 'cymem/__init__.pxd',
 'cymem/cymem.cpython-37m-x86_64-linux-gnu.so',
 'cymem/about.py',
 'cymem/tests/test_import.py',
 'cymem-2.0.2.dist-info/METADATA',
 'cymem-2.0.2.dist-info/LICENSE',
 'cymem-2.0.2.dist-info/RECORD',
 'cymem-2.0.2.dist-info/top_level.txt',
 'cymem-2.0.2.dist-info/WHEEL']

In [ ]:
from wheel.metadata import pkginfo_to_metadata

In [ ]:
meta = fwhl.read(f'{fwhl.dist_info_path}/METADATA').decode()

In [ ]:
m = Parser().parsestr(meta)
m.items()

[('Metadata-Version', '2.1'),
 ('Name', 'cymem'),
 ('Version', '2.0.2'),
 ('Summary', 'Manage calls to calloc/free through Cython'),
 ('Home-page', 'https://github.com/explosion/cymem'),
 ('Author', 'Matthew Honnibal'),
 ('Author-email', 'matt@explosion.ai'),
 ('License', 'MIT'),
 ('Platform', 'UNKNOWN'),
 ('Classifier', 'Environment :: Console'),
 ('Classifier', 'Intended Audience :: Developers'),
 ('Classifier', 'Intended Audience :: Science/Research'),
 ('Classifier', 'License :: OSI Approved :: MIT License'),
 ('Classifier', 'Operating System :: POSIX :: Linux'),
 ('Classifier', 'Operating System :: MacOS :: MacOS X'),
 ('Classifier', 'Operating System :: Microsoft :: Windows'),
 ('Classifier', 'Programming Language :: Cython'),
 ('Classifier', 'Programming Language :: Python :: 2.6'),
 ('Classifier', 'Programming Language :: Python :: 2.7'),
 ('Classifier', 'Programming Language :: Python :: 3.3'),
 ('Classifier', 'Programming Language :: Python :: 3.4'),
 ('Classifier', 'Programm

In [ ]:
zip_ref = zipfile.ZipFile(whl,"r")
ls = zip_ref.filelist
[(i,o.filename) for i,o in enumerate(ls)]

[(0, 'cymem/cymem.pxd'),
 (1, 'cymem/cymem.pyx'),
 (2, 'cymem/__init__.py'),
 (3, 'cymem/__init__.pxd'),
 (4, 'cymem/cymem.cpython-37m-x86_64-linux-gnu.so'),
 (5, 'cymem/about.py'),
 (6, 'cymem/tests/test_import.py'),
 (7, 'cymem-2.0.2.dist-info/METADATA'),
 (8, 'cymem-2.0.2.dist-info/LICENSE'),
 (9, 'cymem-2.0.2.dist-info/RECORD'),
 (10, 'cymem-2.0.2.dist-info/top_level.txt'),
 (11, 'cymem-2.0.2.dist-info/WHEEL')]

In [ ]:
fn = ls[11]
f = zip_ref.read(fn)
print(f.decode())
fn.filename

Wheel-Version: 1.0
Generator: bdist_wheel (0.32.1)
Root-Is-Purelib: false
Tag: cp37-cp37m-manylinux1_x86_64




'cymem-2.0.2.dist-info/WHEEL'